# Asset Pricing — Week 1
## Mathematical Foundations and the Core Pricing Framework

---

This notebook covers the material for the first week of the course. It is divided into two parts:

- **Part A — Mathematical Foundations**: Stochastic processes, Itô's lemma, and dynamic programming.
- **Part B — The Core Pricing Framework**: Arrow-Debreu economies, the stochastic discount factor, and the fundamental pricing equation.

The approach throughout is to build intuition from simple, concrete examples before stating general results. Every formal result is preceded by an example that motivates it.

**Prerequisites**: Probability theory (measure theory at an intuitive level), multivariate calculus, linear algebra, and basic Python/NumPy.

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.stats import norm
from scipy.linalg import solve

plt.rcParams.update({
    'figure.dpi': 120,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
})

rng = np.random.default_rng(seed=42)

---

# Part A — Mathematical Foundations

---

## 1. Uncertainty and Information

Asset pricing is, at its core, the theory of how uncertainty is valued. Before we can price anything, we need a precise language for describing how uncertainty evolves over time.

### 1.1 The Probability Space

We work on a probability space $(\Omega, \mathcal{F}, P)$:

- $\Omega$ is the **sample space** — the set of all possible states of the world.
- $\mathcal{F}$ is a **sigma-algebra** on $\Omega$ — a collection of events that we can assign probabilities to.
- $P$ is a **probability measure** on $(\Omega, \mathcal{F})$ — a function assigning probabilities to events.

**Example**: Consider an economy that lasts two periods (today and tomorrow). Tomorrow, output can be either High ($H$) or Low ($L$). Then:
$$
\Omega = \{H, L\}, \quad \mathcal{F} = \{\emptyset, \{H\}, \{L\}, \Omega\}, \quad P(H) = \pi, \quad P(L) = 1 - \pi.
$$

This is the simplest possible model of uncertainty. Everything we do this week generalises it.

### 1.2 Filtrations and Information

In a dynamic setting, agents learn about the state of the world gradually. A **filtration** $\{\mathcal{F}_t\}_{t \geq 0}$ is a family of sigma-algebras indexed by time, with $\mathcal{F}_s \subseteq \mathcal{F}_t$ whenever $s \leq t$. Intuitively, $\mathcal{F}_t$ is the set of all events that are knowable by time $t$.

A process $\{X_t\}$ is **adapted** to $\{\mathcal{F}_t\}$ if $X_t$ is $\mathcal{F}_t$-measurable for all $t$: its value at time $t$ is known given the information available at $t$. Asset prices must be adapted — you cannot know tomorrow's price today.

---

## 2. Stochastic Processes

A **stochastic process** is a collection of random variables $\{X_t\}_{t \geq 0}$ indexed by time. We distinguish two settings:

- **Discrete time**: $t \in \{0, 1, 2, \ldots\}$. Most empirical work and many theoretical models.
- **Continuous time**: $t \in [0, T]$. Leads to elegant closed-form results (Black-Scholes, affine term structure models).

We begin with discrete time and then transition to continuous time.

### 2.1 Martingales

A process $\{M_t\}$ is a **martingale** with respect to $\{\mathcal{F}_t\}$ and $P$ if:

1. $M_t$ is $\mathcal{F}_t$-measurable (adapted).
2. $E[|M_t|] < \infty$ for all $t$.
3. $E[M_{t+1} \mid \mathcal{F}_t] = M_t$ for all $t$.

Condition 3 says that the best forecast of a martingale's future value, given current information, is its current value. Martingales formalise the notion of a "fair game" — there is no systematic drift.

**Example — Random Walk**: Let $\{\epsilon_t\}$ be i.i.d. with $E[\epsilon_t] = 0$. Define $S_t = \sum_{i=1}^{t} \epsilon_i$ with $S_0 = 0$. Then:
$$
E[S_{t+1} \mid \mathcal{F}_t] = E[S_t + \epsilon_{t+1} \mid \mathcal{F}_t] = S_t + E[\epsilon_{t+1}] = S_t.
$$
So $\{S_t\}$ is a martingale. As we will see, discounted asset prices are martingales under the risk-neutral measure — a central fact in derivatives pricing.

In [ ]:
# Simulate random walks to illustrate the martingale property
T = 100
n_paths = 5

eps = rng.standard_normal((T, n_paths))
S = np.vstack([np.zeros(n_paths), np.cumsum(eps, axis=0)])

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Panel 1: sample paths
ax = axes[0]
for j in range(n_paths):
    ax.plot(S[:, j], lw=1.2, alpha=0.8)
ax.axhline(0, color='black', lw=0.8, ls='--')
ax.set_xlabel('Time $t$')
ax.set_ylabel('$S_t$')
ax.set_title('Sample Paths of a Random Walk\n(martingale: $E[S_{t+1} \mid \\mathcal{F}_t] = S_t$)')

# Panel 2: cross-sectional mean stays near 0 (large N)
n_large = 2000
eps_large = rng.standard_normal((T, n_large))
S_large = np.vstack([np.zeros(n_large), np.cumsum(eps_large, axis=0)])
ax2 = axes[1]
ax2.plot(S_large.mean(axis=1), color='steelblue', lw=1.5, label='Cross-sectional mean')
ax2.fill_between(
    range(T+1),
    S_large.mean(axis=1) - 2*S_large.std(axis=1)/np.sqrt(n_large),
    S_large.mean(axis=1) + 2*S_large.std(axis=1)/np.sqrt(n_large),
    alpha=0.2, color='steelblue', label='±2 s.e.'
)
ax2.axhline(0, color='black', lw=0.8, ls='--')
ax2.set_xlabel('Time $t$')
ax2.set_ylabel('Mean $S_t$ across paths')
ax2.set_title(f'Average over {n_large} paths\nconfirming $E[S_t]=0$ for all $t$')
ax2.legend()

plt.tight_layout()
plt.show()

### 2.2 Brownian Motion

In continuous time, the fundamental building block of randomness is **standard Brownian motion** (also called a Wiener process). A process $\{W_t\}_{t \geq 0}$ is a standard Brownian motion if:

1. $W_0 = 0$ almost surely.
2. **Independent increments**: for $0 \leq s < t$, the increment $W_t - W_s$ is independent of $\mathcal{F}_s$.
3. **Gaussian increments**: $W_t - W_s \sim \mathcal{N}(0, t - s)$.
4. **Continuous paths**: $t \mapsto W_t$ is continuous almost surely.

**Key properties**:
- $E[W_t] = 0$ for all $t$ (it is a martingale).
- $\text{Var}(W_t) = t$ — uncertainty grows linearly with time.
- The paths are *nowhere differentiable* — Brownian motion is very rough.

The roughness of Brownian paths is not a curiosity. It is the mathematical reason why the usual rules of calculus break down and must be replaced by **Itô calculus**.

In [ ]:
# Simulate Brownian motion as the limit of scaled random walks
T_end = 1.0
resolutions = [10, 100, 1000]
colors = ['#e74c3c', '#2980b9', '#27ae60']

fig, axes = plt.subplots(1, 3, figsize=(14, 4), sharey=True)

for ax, N, color in zip(axes, resolutions, colors):
    dt = T_end / N
    dW = rng.standard_normal(N) * np.sqrt(dt)
    W = np.concatenate([[0], np.cumsum(dW)])
    t = np.linspace(0, T_end, N + 1)
    ax.plot(t, W, lw=0.8, color=color)
    ax.set_xlabel('Time $t$')
    ax.set_title(f'$N = {N}$ steps\n$\\Delta t = {dt:.4f}$')
    ax.fill_between(t, -2*np.sqrt(t), 2*np.sqrt(t), alpha=0.08, color='black',
                    label='$\\pm 2\\sqrt{t}$ band')

axes[0].set_ylabel('$W_t$')
axes[0].legend(fontsize=9)
fig.suptitle('Brownian Motion: Finer Grids Reveal Infinite Roughness', fontsize=12)
plt.tight_layout()
plt.show()

print("Note: As N increases, paths become rougher but stay within the ±2√t band.")
print("The variance of W_t equals t — uncertainty scales with the square root of time.")

---

## 3. Itô Calculus

### 3.1 Why Ordinary Calculus Fails

Recall that in ordinary calculus, if $f$ is smooth and $x(t)$ is a differentiable function of time, then by the chain rule:
$$
df(x) = f'(x) \, dx.
$$
We would like a similar rule for functions of Brownian motion. The problem is that Brownian paths are not differentiable, so $dW_t$ does not behave like an ordinary differential.

The key difference is captured by the **quadratic variation** of Brownian motion. Over a small interval $[t, t + dt]$:
$$
(dW_t)^2 = dt \quad (\text{in a mean-square sense}).
$$
In ordinary calculus, $(dx)^2$ is of order $(dt)^2 \to 0$ and is discarded. But for Brownian motion, $(dW)^2$ is of order $dt$ — the same order as the drift. This is why a correction term appears in Itô's lemma.

**The Itô table** (rules for multiplying differentials):

| $\times$ | $dt$ | $dW_t$ |
|----------|------|--------|
| $dt$     | 0    | 0      |
| $dW_t$   | 0    | $dt$   |

### 3.2 Itô Processes

An **Itô process** is a process of the form:
$$
dX_t = \mu_t \, dt + \sigma_t \, dW_t,
$$
where $\mu_t$ (the **drift**) and $\sigma_t$ (the **diffusion**) are adapted processes. The drift is the predictable component; the diffusion is the unpredictable shock.

**Example — Geometric Brownian Motion (GBM)**:
$$
dS_t = \mu S_t \, dt + \sigma S_t \, dW_t,
$$
with $\mu, \sigma > 0$ constants. This is the standard model for stock prices. The drift $\mu$ represents the expected return; $\sigma$ represents volatility.

### 3.3 Itô's Lemma

Let $X_t$ be an Itô process with $dX_t = \mu_t \, dt + \sigma_t \, dW_t$, and let $f(t, x)$ be twice continuously differentiable. Then:

$$
\boxed{df(t, X_t) = \left(f_t + \mu_t f_x + \frac{1}{2}\sigma_t^2 f_{xx}\right) dt + \sigma_t f_x \, dW_t}
$$

The extra term $\frac{1}{2}\sigma_t^2 f_{xx}$ relative to the chain rule arises precisely because $(dW)^2 = dt$.

**Derivation sketch**: Write the Taylor expansion:
$$
df = f_t \, dt + f_x \, dX + \frac{1}{2} f_{xx} \, (dX)^2 + \ldots
$$
Substitute $dX = \mu \, dt + \sigma \, dW$ and apply the Itô table:
$$
(dX)^2 = (\mu \, dt + \sigma \, dW)^2 = \mu^2 (dt)^2 + 2\mu\sigma \, dt \, dW + \sigma^2 (dW)^2 = \sigma^2 \, dt.
$$
The result follows.

### 3.4 Applying Itô's Lemma: GBM Solution

**Problem**: Solve $dS_t = \mu S_t \, dt + \sigma S_t \, dW_t$ with $S_0 > 0$.

**Solution**: Define $f(t, S) = \ln S$. Then $f_t = 0$, $f_S = 1/S$, $f_{SS} = -1/S^2$. By Itô's lemma:
$$
d(\ln S_t) = \frac{1}{S_t} \cdot \mu S_t \, dt + \frac{1}{S_t} \cdot \sigma S_t \, dW_t + \frac{1}{2}\left(-\frac{1}{S_t^2}\right) \sigma^2 S_t^2 \, dt
= \left(\mu - \frac{\sigma^2}{2}\right) dt + \sigma \, dW_t.
$$
This is an ordinary Itô process with constant coefficients. Integrating:
$$
\ln S_t = \ln S_0 + \left(\mu - \frac{\sigma^2}{2}\right)t + \sigma W_t,
$$
so:
$$
\boxed{S_t = S_0 \exp\left[\left(\mu - \frac{\sigma^2}{2}\right)t + \sigma W_t\right].}
$$

**Remark**: The term $-\sigma^2/2$ is the **Itô correction**. It arises because the continuously compounded return ($\ln S_t/S_0$) has a lower mean than the instantaneous return $\mu$. This distinction is important in empirical work — log returns are smaller on average than simple returns by $\sigma^2/2$.

In [ ]:
# Verify the GBM solution numerically via Euler-Maruyama simulation
mu = 0.08     # expected return (annualised)
sigma = 0.20  # volatility (annualised)
S0 = 100.0
T_end = 1.0   # 1 year
N = 252       # daily steps
n_paths = 10_000

dt = T_end / N
dW = rng.standard_normal((N, n_paths)) * np.sqrt(dt)

# Euler-Maruyama discretisation
S = np.empty((N + 1, n_paths))
S[0] = S0
for i in range(N):
    S[i+1] = S[i] * (1 + mu * dt + sigma * dW[i])

# Exact solution
W_cumulative = np.cumsum(dW, axis=0)
t_grid = np.linspace(dt, T_end, N)
S_exact = S0 * np.exp((mu - 0.5 * sigma**2) * t_grid[:, None] + sigma * W_cumulative)

# Compare final distributions
S_T_euler = S[-1]
S_T_exact = S_exact[-1]

fig, axes = plt.subplots(1, 2, figsize=(13, 4))

# Panel 1: a few sample paths
ax = axes[0]
t_full = np.linspace(0, T_end, N + 1)
for j in range(8):
    ax.plot(t_full, S[:, j], lw=1, alpha=0.7)
ax.set_xlabel('Time $t$ (years)')
ax.set_ylabel('Price $S_t$')
ax.set_title(f'GBM Sample Paths\n$\\mu={mu}$, $\\sigma={sigma}$, $S_0={S0}$')

# Panel 2: distribution of S_T — compare Euler vs exact
ax2 = axes[1]
bins = np.linspace(0, 400, 60)
ax2.hist(S_T_euler, bins=bins, alpha=0.5, density=True, label='Euler-Maruyama', color='#e74c3c')
ax2.hist(S_T_exact, bins=bins, alpha=0.5, density=True, label='Exact solution', color='#2980b9')

# Theoretical lognormal density
x = np.linspace(1, 400, 500)
log_mean = np.log(S0) + (mu - 0.5*sigma**2) * T_end
log_std = sigma * np.sqrt(T_end)
pdf = norm.pdf(np.log(x), loc=log_mean, scale=log_std) / x
ax2.plot(x, pdf, 'k-', lw=2, label='Theoretical lognormal')

ax2.set_xlabel('$S_T$')
ax2.set_ylabel('Density')
ax2.set_title('Distribution of $S_T$ at $T=1$')
ax2.legend()

plt.tight_layout()
plt.show()

print(f"Theoretical E[S_T] = {S0 * np.exp(mu * T_end):.2f}")
print(f"Simulated  E[S_T] = {S_T_exact.mean():.2f}")
print(f"\nNote: E[S_T] = S_0 * exp(μT) — the Itô correction cancels in the expectation.")
print(f"Mean log-return: theoretical {(mu - 0.5*sigma**2):.4f}, simulated {np.log(S_T_exact/S0).mean():.4f}")

---

## 4. Dynamic Programming and the Hamilton-Jacobi-Bellman Equation

Most asset pricing models involve agents who make decisions over time to maximise expected utility. The mathematical tool for solving these problems is **dynamic programming**.

### 4.1 The Discrete-Time Problem

An agent chooses a sequence of actions $\{a_t\}$ to solve:
$$
\max_{\{a_t\}} E\left[\sum_{t=0}^{T} \beta^t u(c_t)\right],
$$
where $\beta \in (0,1)$ is a discount factor, $u$ is a utility function, and $c_t$ is consumption at $t$.

**Bellman's principle of optimality** states: An optimal policy has the property that, whatever the initial state and initial decision, the remaining decisions constitute an optimal policy with respect to the state resulting from the first decision.

This leads to the **Bellman equation**:
$$
V_t(x_t) = \max_{a_t} \left\{ u(c_t(x_t, a_t)) + \beta E_t[V_{t+1}(x_{t+1})] \right\},
$$
where $V_t(x_t)$ is the **value function** — the maximum achievable utility from state $x_t$ at time $t$ onwards.

### 4.2 A Worked Example: Optimal Savings

**Setup**: An agent has wealth $W_t$, earns a gross return $R$ on savings, and can consume $c_t \in [0, W_t]$. The budget constraint is:
$$
W_{t+1} = (W_t - c_t) R.
$$
The agent maximises $E\left[\sum_{t=0}^{T-1} \beta^t \ln(c_t)\right]$ (log utility).

**Solution via Bellman equation**:

Guess that $V_t(W) = A_t \ln W + B_t$ for constants $A_t, B_t$. The Bellman equation gives:
$$
A_t \ln W + B_t = \max_{c} \left\{\ln c + \beta (A_{t+1} \ln((W-c)R) + B_{t+1})\right\}.
$$
Taking the first-order condition in $c$:
$$
\frac{1}{c} = \frac{\beta A_{t+1}}{W - c} \implies c^* = \frac{W}{1 + \beta A_{t+1}}.
$$
Substituting back and matching coefficients yields the **optimal consumption rule**:
$$
c_t^* = (1 - \beta) W_t \quad \text{(in the infinite-horizon limit)}.
$$
The agent consumes a fixed fraction $(1-\beta)$ of wealth each period. This is the **permanent income** result for log utility.

In [ ]:
# Solve the finite-horizon savings problem by backward induction
beta = 0.96
R = 1.05
T_horiz = 30  # periods
W0 = 100.0

# Compute the optimal consumption fraction c_t / W_t for each period
# In the finite horizon problem with log utility:
# A_T = 1 (at terminal period, consume everything)
# A_t = 1 + beta * A_{t+1}  (recursive formula)
# c_t^* = W_t / A_t

A = np.zeros(T_horiz + 1)
A[T_horiz] = 1.0  # at the last period, must consume all wealth
for t in range(T_horiz - 1, -1, -1):
    A[t] = 1 + beta * A[t + 1]

consumption_fraction = 1.0 / A  # c_t / W_t

# Simulate a path
W = np.zeros(T_horiz + 1)
c = np.zeros(T_horiz)
W[0] = W0
for t in range(T_horiz):
    c[t] = W[t] * consumption_fraction[t]
    W[t+1] = (W[t] - c[t]) * R

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

ax = axes[0]
ax.plot(range(T_horiz + 1), consumption_fraction[:T_horiz+1], 'o-', ms=4, color='steelblue')
ax.axhline(1 - beta, color='tomato', ls='--', lw=1.5, label=f'Infinite-horizon limit: $1-\\beta={1-beta:.2f}$')
ax.set_xlabel('Period $t$')
ax.set_ylabel('$c_t^* / W_t$')
ax.set_title('Optimal Consumption Fraction Over the Life Cycle')
ax.legend()

ax2 = axes[1]
ax2.plot(range(T_horiz), c, label='Consumption $c_t$', color='steelblue')
ax2.plot(range(T_horiz + 1), W, label='Wealth $W_t$', color='tomato', ls='--')
ax2.set_xlabel('Period $t$')
ax2.set_ylabel('Value')
ax2.set_title(f'Optimal Path ($W_0={W0}$, $\\beta={beta}$, $R={R}$)')
ax2.legend()

plt.tight_layout()
plt.show()

print(f"Infinite-horizon consumption fraction (1-beta): {1-beta:.4f}")
print(f"Period-0 consumption fraction (finite horizon): {consumption_fraction[0]:.4f}")
print("The agent consumes more early in the finite-horizon case (no future to save for).")

### 4.3 The Hamilton-Jacobi-Bellman Equation

In continuous time, the Bellman equation becomes a **partial differential equation** called the **Hamilton-Jacobi-Bellman (HJB) equation**.

Consider an agent maximising $E\left[\int_0^T e^{-\rho t} u(c_t) \, dt\right]$ subject to a wealth process driven by:
$$
dW_t = (\mu(W_t, a_t) - c_t) \, dt + \sigma(W_t, a_t) \, dW_t,
$$
where $a_t$ is the portfolio weight. The value function $V(t, W)$ satisfies:

$$
\boxed{\rho V = \max_{c, a} \left\{ u(c) + V_t + (\mu(W, a) - c) V_W + \frac{1}{2}\sigma(W,a)^2 V_{WW} \right\}.}
$$

The term $\frac{1}{2}\sigma^2 V_{WW}$ is the continuous-time analogue of the $\frac{1}{2}\sigma^2 f_{xx}$ in Itô's lemma — it captures the effect of risk on the value function.

**Key insight**: The HJB equation is a necessary condition for optimality. The Merton (1969) portfolio problem — choose between a risky stock (GBM) and a riskless bond to maximise utility of terminal wealth — is solved precisely using this equation, yielding the famous result that the optimal risky share equals $\frac{\mu - r}{\gamma \sigma^2}$ (where $\gamma$ is the coefficient of relative risk aversion).

---

## 5. Change of Measure and the Girsanov Theorem

This is perhaps the most important mathematical tool in derivatives pricing. It allows us to change the probability measure under which we work, transforming a process with a drift into a martingale.

### 5.1 Motivation

Under the physical measure $P$, the stock price follows:
$$
dS_t = \mu S_t \, dt + \sigma S_t \, dW_t.
$$
The drift $\mu$ reflects the expected return that investors require as compensation for risk. For pricing derivatives, however, we want to remove this drift: if we can find a measure $Q$ under which discounted prices are martingales, pricing becomes simply taking expectations.

### 5.2 The Radon-Nikodym Derivative

A change of measure from $P$ to $Q$ is defined by a **Radon-Nikodym derivative** $\frac{dQ}{dP}$, a positive random variable with $E^P\left[\frac{dQ}{dP}\right] = 1$. For any event $A$:
$$
Q(A) = E^P\left[\frac{dQ}{dP} \cdot \mathbf{1}_A\right].
$$

### 5.3 Girsanov's Theorem

Let $\theta_t$ be an adapted process (the **market price of risk**). Define:
$$
\tilde{W}_t = W_t + \int_0^t \theta_s \, ds.
$$
Under the measure $Q$ defined by the Radon-Nikodym derivative:
$$
\frac{dQ}{dP}\bigg|_{\mathcal{F}_t} = \exp\left(-\int_0^t \theta_s \, dW_s - \frac{1}{2}\int_0^t \theta_s^2 \, ds\right),
$$
the process $\{\tilde{W}_t\}$ is a standard Brownian motion.

**Application**: Choose $\theta = (\mu - r)/\sigma$ (the Sharpe ratio). Under $Q$:
$$
dS_t = r S_t \, dt + \sigma S_t \, d\tilde{W}_t.
$$
The drift has changed from $\mu$ to $r$. Under $Q$, the discounted stock price $e^{-rt} S_t$ is a martingale. This measure $Q$ is called the **risk-neutral measure** (or equivalent martingale measure).

> **The fundamental insight**: Changing the measure shifts the drift without changing the volatility. The market price of risk $\theta = (\mu-r)/\sigma$ measures how many units of volatility investors are compensated for per unit of drift above the riskfree rate.

We will see in Part B that this is intimately connected to the stochastic discount factor.

In [ ]:
# Illustrate the change of measure: P vs Q
mu_P = 0.10   # expected return under P
r = 0.02      # risk-free rate
sigma_gbm = 0.20
T_end = 1.0
N = 252
n_paths = 50_000
dt = T_end / N

# Generate Brownian increments under P
dW_P = rng.standard_normal((N, n_paths)) * np.sqrt(dt)

# Market price of risk
theta = (mu_P - r) / sigma_gbm

# W_tilde = W + theta*t  is BM under Q
# Under Q: dS = r*S*dt + sigma*S*dW_tilde
# Simulate S under P and Q
log_S_T_P = (mu_P - 0.5*sigma_gbm**2)*T_end + sigma_gbm * np.sum(dW_P, axis=0)
log_S_T_Q = (r - 0.5*sigma_gbm**2)*T_end + sigma_gbm * np.sum(dW_P, axis=0)  # same Brownian paths

S_T_P = np.exp(log_S_T_P)  # starting from S0=1
S_T_Q = np.exp(log_S_T_Q)

fig, axes = plt.subplots(1, 2, figsize=(13, 4))

ax = axes[0]
bins = np.linspace(0.2, 2.5, 80)
ax.hist(S_T_P, bins=bins, density=True, alpha=0.5, color='#e74c3c', label=f'Under $P$ ($\\mu={mu_P}$)')
ax.hist(S_T_Q, bins=bins, density=True, alpha=0.5, color='#2980b9', label=f'Under $Q$ ($r={r}$)')
ax.axvline(S_T_P.mean(), color='#e74c3c', lw=2, ls='--', label=f'$E^P[S_T]={S_T_P.mean():.3f}$')
ax.axvline(S_T_Q.mean(), color='#2980b9', lw=2, ls='--', label=f'$E^Q[S_T]={S_T_Q.mean():.3f}$')
ax.set_xlabel('$S_T$ (with $S_0=1$)')
ax.set_ylabel('Density')
ax.set_title('Distribution of $S_T$ Under $P$ vs $Q$')
ax.legend(fontsize=9)

ax2 = axes[1]
# Show that e^{-rT} * E^Q[S_T] = S_0 = 1 (no-arbitrage / martingale check)
discounted_Q = np.exp(-r * T_end) * S_T_Q
discounted_P = np.exp(-r * T_end) * S_T_P
ax2.hist(discounted_Q, bins=bins, density=True, alpha=0.6, color='#2980b9',
         label=f'$e^{{-rT}}S_T$ under $Q$: mean={discounted_Q.mean():.4f}')
ax2.axvline(1.0, color='black', lw=2, ls='-', label='$S_0=1$ (no-arbitrage)')
ax2.axvline(discounted_Q.mean(), color='#2980b9', lw=2, ls='--')
ax2.set_xlabel('$e^{-rT} S_T$')
ax2.set_ylabel('Density')
ax2.set_title('Discounted $S_T$ Under $Q$\nMean Should Equal $S_0=1$')
ax2.legend(fontsize=9)

plt.tight_layout()
plt.show()

print(f"Market price of risk: θ = (μ-r)/σ = ({mu_P}-{r})/{sigma_gbm} = {theta:.2f}")
print(f"E^Q[e^(-rT) S_T] = {discounted_Q.mean():.4f} ≈ S_0 = 1.0 ✓")

---

# Part B — The Core Pricing Framework

---

## 6. Arrow-Debreu Economies

We now build up the theory of asset pricing from first principles, beginning with the simplest possible economy.

### 6.1 The Two-Period, Two-State Economy

**Setup**: There are two dates, $t = 0$ (today) and $t = 1$ (tomorrow). At $t=1$, the world will be in one of two states: $s = 1$ ("good") or $s = 2$ ("bad"), with probabilities $\pi_1$ and $\pi_2 = 1 - \pi_1$.

An **Arrow-Debreu security** for state $s$ pays \$1 if state $s$ occurs and \$0 otherwise. Let $q_s$ be the price of this security at $t=0$.

**Example**: Suppose $\pi_1 = 0.6$, $\pi_2 = 0.4$, and the Arrow-Debreu prices are $q_1 = 0.5$ and $q_2 = 0.45$.

Consider two assets:
- **Riskless bond**: pays 1 in both states. Price = $q_1 + q_2 = 0.5 + 0.45 = 0.95$. Gross return $R_f = 1/0.95 \approx 1.053$.
- **Risky stock**: pays $d_1 = 2$ in state 1, $d_2 = 0.5$ in state 2. Price = $2 \times 0.5 + 0.5 \times 0.45 = 1.0 + 0.225 = 1.225$.

**Key observation**: If we know the Arrow-Debreu prices $\{q_s\}$, we can price *any* asset as $p = \sum_s q_s x_s$, where $x_s$ is the asset's payoff in state $s$.

### 6.2 No-Arbitrage and the Positivity of Arrow-Debreu Prices

**Definition (Arbitrage)**: An arbitrage is a portfolio that costs zero (or less) at $t=0$ and pays a non-negative amount in every state at $t=1$, with a strictly positive payment in at least one state.

**Theorem**: There is no arbitrage if and only if there exist strictly positive Arrow-Debreu prices $q_s > 0$ for all $s$.

**Intuition**: If some $q_s \leq 0$, one could buy the Arrow-Debreu security for state $s$ at zero or negative cost and receive a positive payoff — a free lunch.

The positivity of $q_s$ is not a behavioural assumption; it is a consequence of the absence of arbitrage alone.

### 6.3 Risk-Neutral Probabilities

Define $\pi_s^Q = q_s / \sum_{s'} q_{s'}$ — the Arrow-Debreu prices normalised to sum to one. These are called **risk-neutral probabilities** (also called risk-adjusted, equivalent martingale measure probabilities).

Let $R_f = 1/\sum_s q_s$ be the gross riskfree rate. Then for any asset paying $x_s$ in state $s$:
$$
p = \sum_s q_s x_s = \frac{1}{R_f} \sum_s \pi_s^Q x_s = \frac{E^Q[x]}{R_f}.
$$
**Under the risk-neutral measure, all assets have the same expected return — the riskfree rate.** This is why the risk-neutral measure is useful: pricing is simply discounting expected payoffs at the riskfree rate, with no risk premium needed.

In [ ]:
# Arrow-Debreu pricing: concrete example
pi = np.array([0.6, 0.4])  # physical probabilities
q = np.array([0.50, 0.45]) # Arrow-Debreu prices

# Payoff matrix: rows = assets, cols = states
# Asset 1: riskless bond (pays 1 in each state)
# Asset 2: risky stock
# Asset 3: call option on the stock with strike K=1
payoffs = np.array([
    [1.0, 1.0],    # bond
    [2.0, 0.5],    # stock
    [1.0, 0.0],    # call option: max(stock_payoff - K, 0) with K=1
])

# Price each asset: p = q @ payoffs.T  (dot product of AD prices with payoffs)
prices = payoffs @ q

# Risk-free rate
Rf = 1 / q.sum()

# Risk-neutral probabilities
pi_Q = q / q.sum()

# Expected returns under P and Q
print("=" * 55)
print(f"Arrow-Debreu prices: q = {q}")
print(f"Physical probs:      π = {pi}")
print(f"Risk-neutral probs:  πᵩ = {pi_Q.round(4)}")
print(f"Riskfree gross return: Rf = {Rf:.4f}")
print("=" * 55)

asset_names = ['Bond', 'Stock', 'Call Option (K=1)']
for i, name in enumerate(asset_names):
    p = prices[i]
    E_P = pi @ payoffs[i]          # expected payoff under P
    E_Q = pi_Q @ payoffs[i]        # expected payoff under Q
    ret_P = E_P / p                # expected gross return under P
    ret_Q = E_Q / p                # expected gross return under Q
    print(f"\n{name}:")
    print(f"  Payoffs: {payoffs[i]}")
    print(f"  Price = {p:.4f}")
    print(f"  E^P[x]/p = {ret_P:.4f}   (expected gross return under P)")
    print(f"  E^Q[x]/p = {ret_Q:.4f}   (should equal Rf = {Rf:.4f})")

---

## 7. The Stochastic Discount Factor

The Arrow-Debreu framework is elegant but requires a price for every state of the world — which is impractical when the state space is large. The **stochastic discount factor** (SDF) provides a more compact representation of the same idea.

### 7.1 From Arrow-Debreu Prices to the SDF

Recall that $p = \sum_s q_s x_s$. Define:
$$
M_s \equiv \frac{q_s}{\pi_s} \quad \text{(the state price per unit of probability)}.
$$
Then:
$$
p = \sum_s \pi_s M_s x_s = E[Mx].
$$
The random variable $M$ (equal to $M_s$ in state $s$) is the **stochastic discount factor** or **pricing kernel**. In continuous time, we write it as a process $\{M_t\}$.

The **fundamental pricing equation** is:
$$
\boxed{p_t = E_t[M_{t+1} x_{t+1}]}
$$
or equivalently, dividing by $p_t$ to express in terms of gross returns $R_{t+1} = x_{t+1}/p_t$:
$$
\boxed{E_t[M_{t+1} R_{t+1}] = 1.}
$$

**This single equation is the foundation of the entire field of asset pricing.** Every model we will study — CAPM, CCAPM, APT, Black-Scholes — can be derived as a special case by specifying what $M$ looks like.

### 7.2 Properties of the SDF

**Property 1 — Riskfree rate**: Applied to a riskfree asset with return $R_f$:
$$
E[M R_f] = R_f E[M] = 1 \implies E[M] = \frac{1}{R_f}.
$$
The expected value of the SDF is the riskfree discount factor.

**Property 2 — Risk premium**: Subtract the riskfree pricing equation from the general one:
$$
E[R] - R_f = -R_f \, \text{Cov}(M, R).
$$
This is the central equation of modern asset pricing. The **risk premium** of any asset is determined by the covariance of its return with the SDF. Assets that covary *negatively* with $M$ (i.e., they pay poorly in bad states) require a positive risk premium. Assets that covary positively with $M$ (hedges) can have returns below $R_f$.

**Derivation**: Since $E[MR] = 1 = E[M]R_f$:
$$
E[M](E[R] - R_f) = -\text{Cov}(M, R) \implies E[R] - R_f = -\frac{\text{Cov}(M,R)}{E[M]} = -R_f \, \text{Cov}(M,R).
$$

In [ ]:
# Illustrate: risk premium = -Rf * Cov(M, R)
# Use the Arrow-Debreu example above
pi = np.array([0.6, 0.4])
q = np.array([0.50, 0.45])

M = q / pi       # SDF values in each state
Rf = 1 / q.sum()

print("SDF values by state:")
for s in range(2):
    print(f"  State {s+1}: M = q/π = {q[s]:.2f}/{pi[s]:.2f} = {M[s]:.4f}")

print(f"\nE[M] = {(pi @ M):.4f} = 1/Rf = {1/Rf:.4f} ✓")

# Verify risk premium formula for the stock
stock_payoffs = np.array([2.0, 0.5])
stock_price = q @ stock_payoffs
stock_returns = stock_payoffs / stock_price  # gross returns by state

ER = pi @ stock_returns         # E^P[R]
CovMR = pi @ (M * stock_returns) - (pi @ M) * (pi @ stock_returns)
risk_premium_formula = -Rf * CovMR
risk_premium_direct = ER - Rf

print(f"\nStock gross returns by state: {stock_returns.round(4)}")
print(f"E^P[R_stock] = {ER:.4f}")
print(f"Cov(M, R_stock) = {CovMR:.4f}")
print(f"\nRisk premium (direct):  E[R] - Rf = {risk_premium_direct:.4f}")
print(f"Risk premium (formula): -Rf*Cov(M,R) = {risk_premium_formula:.4f} ✓")
print(f"\nThe stock pays well (high return) in state 1 (good state)")
print(f"but M is LOW in state 1 (cheap insurance, no scarcity).")
print(f"=> Cov(M, R) < 0 => positive risk premium required.")

### 7.3 The SDF in Continuous Time

In continuous time, the pricing equation becomes:
$$
p_t = E_t\left[\int_t^T M_s c_s \, ds + M_T P_T\right],
$$
where $c_s$ are dividends paid between $t$ and $T$, and $P_T$ is the terminal price.

Equivalently, $M_t p_t + \int_0^t M_s c_s \, ds$ must be a martingale (no-arbitrage condition in continuous time).

The SDF process typically takes the form:
$$
\frac{dM_t}{M_t} = -r_t \, dt - \theta_t \, dW_t,
$$
where $r_t$ is the instantaneous riskfree rate and $\theta_t$ is the **market price of risk**. This connects directly to Girsanov's theorem: $\theta_t$ in the SDF is the same $\theta_t$ that appears in the change of measure.

**Observation**: The SDF has a *negative* drift ($-r_t$) — it discounts future payoffs — and a *negative* loading on $dW_t$. Because $W_t$ represents positive shocks to the economy, and good states have low marginal utility, the SDF falls in good states and rises in bad states. This negative covariance is what generates positive risk premia.

---

## 8. The Hansen-Jagannathan Bound

We have established that the SDF exists (under no-arbitrage) but said little about what it looks like. The **Hansen-Jagannathan (HJ) bound** provides a lower bound on the volatility of any valid SDF, derived from observable asset returns alone.

### 8.1 Derivation

From the risk premium equation:
$$
E[R^i] - R_f = -R_f \, \text{Cov}(M, R^i) = -R_f \, \text{Cor}(M, R^i) \cdot \text{std}(M) \cdot \text{std}(R^i).
$$
Since $|\text{Cor}(M, R^i)| \leq 1$:
$$
|E[R^i] - R_f| \leq R_f \cdot \text{std}(M) \cdot \text{std}(R^i).
$$
Rearranging:
$$
\boxed{\frac{\text{std}(M)}{E[M]} \geq \frac{|E[R^i] - R_f|}{\text{std}(R^i)} = |\text{Sharpe ratio of asset } i|.}
$$
The **coefficient of variation of the SDF must be at least as large as the maximum Sharpe ratio** achievable from the asset menu.

### 8.2 Empirical Implications

The equity Sharpe ratio for the U.S. stock market over the postwar period is approximately $0.4$ per year (an equity premium of roughly 6% with standard deviation of returns around 15%). The HJ bound therefore requires:
$$
\frac{\text{std}(M)}{E[M]} \geq 0.4.
$$
This is a very strong requirement. The consumption-based model (CCAPM) with CRRA utility implies $M_{t+1} = \beta (C_{t+1}/C_t)^{-\gamma}$. With $\text{std}(\Delta \ln C) \approx 0.02$ (historically), one needs $\gamma \approx 20$ to satisfy the bound. But $\gamma \approx 20$ implies a riskfree rate far higher than observed — the **equity premium puzzle** and the **riskfree rate puzzle**.

This is the central challenge that Modules 2 and 3 of the course will attempt to address.

In [ ]:
# Visualise the Hansen-Jagannathan bound
# Show: the CCAPM with power utility barely enters the feasible region

# Empirical moments (approximate US postwar annual data)
E_Rm_minus_Rf = 0.06    # equity premium
std_Rm = 0.15           # stock return vol
sharpe_equity = E_Rm_minus_Rf / std_Rm

# The HJ bound: std(M)/E[M] >= Sharpe ratio
# In mean-std space of M: E[M] = 1/Rf, so we plot std(M) vs E[M]
# The frontier: std(M) = |Sharpe| * E[M]  (a ray from origin)

E_M_vals = np.linspace(0.85, 1.05, 200)   # range of E[M] = 1/Rf
std_M_bound = sharpe_equity * E_M_vals     # HJ lower bound on std(M)

# CCAPM: M = beta * (C_{t+1}/C_t)^{-gamma}
# Consumption growth: log(C_{t+1}/C_t) ~ N(mu_c, sigma_c^2)
beta_agent = 0.99
mu_c = 0.018    # mean consumption growth
sigma_c = 0.02  # std of consumption growth (historical)

gammas = np.array([1, 5, 10, 20, 50])
ccapm_points = []
for gamma in gammas:
    # M = beta * exp(-gamma * Delta log C)  
    # log M ~ N(log(beta) - gamma*mu_c, gamma^2 * sigma_c^2)
    log_M_mean = np.log(beta_agent) - gamma * mu_c
    log_M_var  = gamma**2 * sigma_c**2
    E_M   = np.exp(log_M_mean + 0.5 * log_M_var)
    std_M = E_M * np.sqrt(np.exp(log_M_var) - 1)
    ccapm_points.append((E_M, std_M, gamma))

fig, ax = plt.subplots(figsize=(8, 5))

# HJ bound
ax.plot(E_M_vals, std_M_bound, 'k-', lw=2, label=f'HJ Bound (slope = Sharpe = {sharpe_equity:.2f})')
ax.fill_between(E_M_vals, std_M_bound, std_M_bound.max() * 1.5,
                alpha=0.08, color='green', label='Feasible region (above bound)')

# CCAPM points
for (E_M, std_M, gamma) in ccapm_points:
    color = 'red' if std_M < sharpe_equity * E_M else 'green'
    ax.scatter(E_M, std_M, color=color, s=80, zorder=5)
    ax.annotate(f'$\\gamma={gamma}$', (E_M, std_M), textcoords='offset points',
                xytext=(5, 3), fontsize=9)

ax.set_xlabel('$E[M]$ (riskfree discount factor $= 1/R_f$)')
ax.set_ylabel('std$(M)$')
ax.set_title('The Hansen-Jagannathan Bound\nCCAPM must lie above the frontier to be consistent with data')
ax.set_xlim(0.85, 1.05)
ax.set_ylim(0, 0.45)
ax.legend()

plt.tight_layout()
plt.show()

print("Red dots: CCAPM violates the HJ bound (infeasible SDF).")
print("Green dots: CCAPM satisfies the bound.")
print(f"\nWith σ_c = {sigma_c}, one needs γ ≥ {sharpe_equity/sigma_c:.0f} to satisfy the bound.")
print("But such high γ implies implausibly high risk-free rates — the equity premium puzzle.")

---

## 9. The SDF and Factor Models

We have established the SDF framework in its full generality. It is instructive to see how it encompasses familiar models as special cases.

### 9.1 From the SDF to the CAPM

Suppose the SDF is linear in the market return:
$$
M = a + b \, R_m,
$$
where $R_m$ is the gross return on the market portfolio and $a, b$ are constants. Apply $E[MR^i] = 1$:
$$
a E[R^i] + b \, E[R_m R^i] = 1.
$$
Using $E[R_m R^i] = \text{Cov}(R_m, R^i) + E[R_m] E[R^i]$ and the riskfree equation $a + b E[R_m] = 1/R_f \cdot R_f = ...$, one can show:
$$
E[R^i] - R_f = \beta_i (E[R_m] - R_f), \quad \beta_i = \frac{\text{Cov}(R^i, R_m)}{\text{Var}(R_m)}.
$$
This is the **Capital Asset Pricing Model (CAPM)**. It follows from the SDF if and only if $M$ is linear in $R_m$. The CAPM is not a free-standing theory — it is a statement about the form of the pricing kernel.

**When is $M$ linear in $R_m$?** In equilibrium models, this holds when all investors have mean-variance preferences, or when returns are jointly normally distributed with CARA utility. The CAPM rests on these special assumptions.

### 9.2 General Factor Structure

More generally, suppose:
$$
M = a + b_1 f_1 + b_2 f_2 + \ldots + b_K f_K,
$$
where $f_1, \ldots, f_K$ are **factors**. Then:
$$
E[R^i] - R_f = \sum_{k=1}^K \beta_{ik} \lambda_k,
$$
where $\beta_{ik} = \text{Cov}(R^i, f_k)/\text{Var}(f_k)$ is asset $i$'s loading on factor $k$, and $\lambda_k$ is the **factor risk premium**. This is the **Arbitrage Pricing Theory (APT)**.

The Fama-French 3-factor model, the 5-factor model, and the machine learning models we study later are all examples of this structure. The empirical question is: which factors enter $M$? This is the central unresolved question of the field.

---

## 10. Uniqueness of the SDF and Market Completeness

We have shown that an SDF exists under no-arbitrage. A natural question is: is it unique?

**Theorem**: The SDF is unique if and only if markets are **complete** — every possible payoff pattern across states can be replicated by some portfolio of traded assets.

**Proof sketch**: In the two-state economy, two linearly independent assets (e.g., the bond and the stock) span all payoff patterns. There is a unique $(q_1, q_2)$ satisfying the pricing equations, hence a unique $M$.

With three states and only two assets, the system is underdetermined: there are infinitely many $(q_1, q_2, q_3)$ consistent with no-arbitrage. Each corresponds to a different valid SDF.

**Practical implication**: In incomplete markets (the empirically relevant case — we have far more states of the world than assets), the SDF is not uniquely identified from asset prices alone. We need additional structure — an equilibrium model — to pin it down. This is why we spend so much time on equilibrium models in Module 2.

In [ ]:
# Illustrate: complete markets → unique SDF; incomplete → many valid SDFs

# === Complete markets: 3 states, 3 assets ===
# Payoff matrix A: A[i,s] = payoff of asset i in state s
A_complete = np.array([
    [1.0, 1.0, 1.0],   # riskless bond
    [3.0, 1.0, 0.5],   # stock
    [0.0, 0.0, 1.0],   # Arrow-Debreu for state 3
], dtype=float)

prices_complete = np.array([0.95, 1.10, 0.30])  # observed prices
pi_3 = np.array([0.5, 0.3, 0.2])               # physical probabilities

# Solve for Arrow-Debreu prices: A.T @ q = prices
q_complete = solve(A_complete.T, prices_complete)
M_complete = q_complete / pi_3

print("=== Complete Markets (3 states, 3 assets) ===")
print(f"Arrow-Debreu prices: q = {q_complete.round(4)}")
print(f"SDF values:          M = {M_complete.round(4)}")
print(f"All q > 0: {np.all(q_complete > 0)} — no-arbitrage satisfied")
print(f"E[M] = {(pi_3 @ M_complete):.4f}")

# === Incomplete markets: 3 states, 2 assets ===
# The system A[2x3] q[3x1] = p[2x1] is underdetermined
A_incomplete = A_complete[:2, :]  # remove the third asset
prices_incomplete = prices_complete[:2]

# Find a particular solution and the null space
# A_incomplete.T @ q = prices_incomplete  — 2 equations, 3 unknowns
# Parametrise: q = q_particular + t * null_vector

# Minimum-norm solution via least squares
q_particular, _, _, _ = np.linalg.lstsq(A_incomplete.T, prices_incomplete, rcond=None)

# Null space of A_incomplete.T (a 2x3 matrix → 1D null space)
_, _, Vt = np.linalg.svd(A_incomplete.T)
null_vec = Vt[-1]  # last right singular vector

t_vals = np.linspace(-0.3, 0.3, 500)
q_family = q_particular[None, :] + t_vals[:, None] * null_vec[None, :]

# Keep only no-arbitrage (q > 0) solutions
valid = np.all(q_family > 0, axis=1)
q_valid = q_family[valid]
M_valid = q_valid / pi_3[None, :]

print(f"\n=== Incomplete Markets (3 states, 2 assets) ===")
print(f"Number of valid (q>0) SDF candidates: {valid.sum()}")
print(f"std(M) ranges from {M_valid.std(axis=1).min():.4f} to {M_valid.std(axis=1).max():.4f}")
print("=> The SDF is NOT uniquely identified without additional model structure.")

fig, ax = plt.subplots(figsize=(8, 4))
ax.scatter(M_valid[:, 0], M_valid[:, 1], c=M_valid[:, 2], cmap='viridis', s=8, alpha=0.7)
ax.scatter(*M_complete[:2], color='red', s=120, zorder=5, label='Unique SDF (complete markets)')
ax.set_xlabel('$M$ in state 1')
ax.set_ylabel('$M$ in state 2')
ax.set_title('Valid SDFs in Incomplete Markets\n(color = $M$ in state 3)')
ax.legend()
plt.colorbar(ax.collections[0], label='$M$ in state 3')
plt.tight_layout()
plt.show()

---

## 11. Summary and Looking Ahead

This first week has established the two pillars on which the rest of the course rests.

**Mathematical foundations**:
- Stochastic processes formalise how uncertainty evolves over time. Brownian motion is the continuous-time building block.
- Itô's lemma is the chain rule of stochastic calculus, with a correction term ($\frac{1}{2}\sigma^2 f_{xx}$) arising from the non-negligible quadratic variation of Brownian motion.
- Dynamic programming provides a recursive method for solving intertemporal optimisation problems; the Bellman equation and the HJB equation are its discrete- and continuous-time expressions.
- The Girsanov theorem allows us to change the probability measure, removing the drift from a stochastic process and connecting directly to risk-neutral pricing.

**Core pricing framework**:
- Arrow-Debreu securities are the atomic units of a complete market. Their prices fully characterise the market's valuation of risk.
- The stochastic discount factor $M$ encodes all pricing information: $p = E[Mx]$.
- The risk premium of any asset equals $-R_f \, \text{Cov}(M, R)$. Assets that pay poorly in states where the SDF is high (bad times) command high risk premia.
- The Hansen-Jagannathan bound requires the SDF to be sufficiently volatile to explain observed Sharpe ratios. Standard consumption-based models fail this bound — the equity premium puzzle.
- Every major asset pricing model (CAPM, APT, factor models) is a model of $M$.

**In Week 2**, we will ask: where does $M$ come from in equilibrium? We derive $M$ from the utility maximisation of a representative agent, which leads to the consumption CAPM and the classical puzzles.

---

## Problem Set — Week 1

**Problem 1 — Itô's Lemma**: Let $X_t$ follow GBM with drift $\mu$ and volatility $\sigma$. Using Itô's lemma, find the dynamics of $f(X_t) = X_t^n$ for integer $n \geq 2$. Verify that the solution is $X_t^n = X_0^n \exp\left[\left(n\mu + \frac{n(n-1)}{2}\sigma^2\right)t + n\sigma W_t\right]$ (up to a corrected drift).

**Problem 2 — Arrow-Debreu pricing**: An economy has three equally likely states. Two assets trade: a bond (pays 1 in all states, price = 0.97) and a stock (pays 2, 1, 0 in states 1, 2, 3, price = 0.90). (a) Is this market complete? (b) Characterise all no-arbitrage SDFs. (c) What is the range of admissible prices for an Arrow-Debreu security for state 1?

**Problem 3 — Risk premium decomposition**: Suppose $M = 1 - b(R_m - E[R_m])$ where $b > 0$ and $R_m$ is the market return. (a) Show that this SDF implies the CAPM. (b) Express $b$ in terms of $R_f$, $E[R_m]$, and $\text{Var}(R_m)$. (c) What does this SDF imply about the correlation between the SDF and market returns?

**Problem 4 — HJ bound**: Using annual U.S. data from FRED (or simulated data), estimate the equity Sharpe ratio. Compute the implied lower bound on $\text{std}(M)/E[M]$. For the power utility CCAPM with $\gamma \in \{1, 5, 10, 20\}$ and empirically calibrated consumption growth moments, compute $\text{std}(M)/E[M]$ and determine which values of $\gamma$ satisfy the bound.

**Problem 5 — Bellman equation**: Consider the optimal savings problem in Section 4.2, but with CRRA utility $u(c) = c^{1-\gamma}/(1-\gamma)$ and $\gamma \neq 1$. (a) Guess a value function of the form $V(W) = A W^{1-\gamma}/(1-\gamma)$ and solve for $A$. (b) What is the optimal consumption-wealth ratio? (c) How does it depend on $\gamma$? Does higher risk aversion imply more or less savings?

---

## References

- Cochrane, J. H. (2005). *Asset Pricing* (revised ed.). Princeton University Press. — Chapters 1–4.
- Shreve, S. (2004). *Stochastic Calculus for Finance II*. Springer. — Chapters 1–4 (mathematical background).
- Hansen, L. P. and Jagannathan, R. (1991). Implications of security market data for models of dynamic economies. *Journal of Political Economy*, 99(2), 225–262.
- Mehra, R. and Prescott, E. C. (1985). The equity premium: A puzzle. *Journal of Monetary Economics*, 15(2), 145–161.
- Black, F. and Scholes, M. (1973). The pricing of options and corporate liabilities. *Journal of Political Economy*, 81(3), 637–654.